## Working on raw data

1. Working on newly fetched data which contains all the columns provided by the api.
2. **AIM:** By using this data my aim is to extract only **BUILTUP_AREA** or **PLOT_AREA** for every properties. Because previously I was using pre-defined **AREA** column present in the filtered data.

In [1]:
# !pip install arv-easy-analysis

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from easy_analysis.data import summary
from easy_analysis.plots import univariate, multivariate

In [3]:
pd.set_option(
    "display.float_format", lambda x: f"{x:.2f}" if isinstance(x, float) else x
)

In [4]:
df = pd.read_csv("data/v2/gurgaon_4k.csv").query("PREFERENCE!='p'")
print(df.shape)
df.head()

(3432, 226)


,SPID,PROP_ID,PHOTO_URL,MEDIUM_PHOTO_URL,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,LOCALITY,AREA_UNIT,...,BROKER_GROUP,QUALITY_BUCKET,GENUINENESS_BUCKET,DATA_QUALITY_INFO,IS_CREDIT_LISTING,SHOW_ONLY_START_PRICE,OTHER_CITY,FOOD_AVAILABILITY,AVAILABILITY_DATE__U,MODIFY_DATE__U
0,71649778,https://99acres.com/I71649778,NaN,NaN,s,the project offers 2 bhk apartments in sector ...,residential apartment,greater noida,sector 1 greater noida west,sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71227190,https://99acres.com/D71227190,https://mediacdn.99acres.com/media1/22200/13/4...,https://mediacdn.99acres.com/media1/22200/13/4...,s,welcome to signature global city 37d ii. the p...,independent/builder floor,gurgaon,sector 37d gurgaon,sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71649754,https://99acres.com/Z71649754,https://mediacdn.99acres.com/media1/22446/1/44...,https://mediacdn.99acres.com/media1/22446/1/44...,s,sportshome by dev sai group is an under constr...,residential apartment,greater noida,sector 1 greater noida west,sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71380106,https://99acres.com/F71380106,NaN,NaN,s,gls arawali city is a residential development ...,residential land,gurgaon,sohna,sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71374796,https://99acres.com/D71374796,https://mediacdn.99acres.com/media1/22285/3/44...,https://mediacdn.99acres.com/media1/22285/3/44...,s,"own a 3 bhk apartment in express astra, sector...",residential apartment,greater noida,sector 1 greater noida west,sq.ft.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.duplicated("PROP_ID").sum()

0

In [6]:
imp_cols = [
    "PROP_ID",
    "CITY",
    "CLASS_LABEL",
    "PREFERENCE",
    "PROPERTY_TYPE",
    "PRICE",
    "AREA",
    "CARPET_SQFT",
    "SUPERBUILTUP_SQFT",
    "BUILTUP_SQFT",
    "SUPER_SQFT",
]
area_cols = ["AREA", "CARPET_SQFT", "SUPERBUILTUP_SQFT", "BUILTUP_SQFT", "SUPER_SQFT"]
imp_area_cols = area_cols[1:].copy()

## General steps

### Check some important features

In [7]:
df["CITY"].value_counts()

CITY
gurgaon          1770
greater noida     380
noida             368
south delhi       276
faridabad         211
ghaziabad         209
west delhi         77
dwarka delhi       60
east delhi         29
central delhi      27
north delhi        25
Name: count, dtype: int64

In [8]:
df["PREFERENCE"].value_counts()

PREFERENCE
s    3213
r     219
Name: count, dtype: int64

In [9]:
df["PROPERTY_TYPE"].value_counts()

PROPERTY_TYPE
residential apartment        2095
independent/builder floor     899
residential land              270
independent house/villa       168
Name: count, dtype: int64

In [10]:
pd.crosstab(df["PREFERENCE"], df["PROPERTY_TYPE"])

PROPERTY_TYPE,independent house/villa,independent/builder floor,residential apartment,residential land
PREFERENCE,,,,
r,19,61,139,0
s,149,838,1956,270


In [11]:
pd.crosstab(df["CLASS_LABEL"], df["PROPERTY_TYPE"])

PROPERTY_TYPE,independent house/villa,independent/builder floor,residential apartment,residential land
CLASS_LABEL,,,,
builder,0,2,12,0
dealer,128,829,1939,237
owner,40,68,144,33


In [12]:
pd.crosstab(df["CLASS_LABEL"], df["PREFERENCE"])

PREFERENCE,r,s
CLASS_LABEL,,
builder,0,14
dealer,179,2954
owner,40,245


In [13]:
pd.crosstab(df["CITY"], df["CLASS_LABEL"])

CLASS_LABEL,builder,dealer,owner
CITY,,,
central delhi,0,25,2
dwarka delhi,0,49,11
east delhi,0,7,22
faridabad,0,200,11
ghaziabad,3,183,23
greater noida,0,360,20
gurgaon,5,1674,91
noida,2,342,24
north delhi,1,5,19


In [14]:
pd.crosstab(df["CITY"], df["PREFERENCE"])

PREFERENCE,r,s
CITY,,
central delhi,0,27
dwarka delhi,2,58
east delhi,4,25
faridabad,0,211
ghaziabad,11,198
greater noida,11,369
gurgaon,156,1614
noida,18,350
north delhi,1,24


In [15]:
pd.crosstab(df["CITY"], df["PROPERTY_TYPE"])

PROPERTY_TYPE,independent house/villa,independent/builder floor,residential apartment,residential land
CITY,,,,
central delhi,0,1,26,0
dwarka delhi,0,30,29,1
east delhi,3,5,20,1
faridabad,4,111,60,36
ghaziabad,5,32,159,13
greater noida,39,2,304,35
gurgaon,81,451,1074,164
noida,22,0,335,11
north delhi,4,6,13,2


In [16]:
df[area_cols].isnull().sum()

AREA                    0
CARPET_SQFT          2224
SUPERBUILTUP_SQFT    2170
BUILTUP_SQFT         2968
SUPER_SQFT           3134
dtype: int64

In [17]:
df[area_cols].notnull().sum()

AREA                 3432
CARPET_SQFT          1208
SUPERBUILTUP_SQFT    1262
BUILTUP_SQFT          464
SUPER_SQFT            298
dtype: int64

In [18]:
for col in area_cols:
    print(col)
    print(df[df[col].notnull()]["PROPERTY_TYPE"].value_counts())
    print()

AREA
PROPERTY_TYPE
residential apartment        2095
independent/builder floor     899
residential land              270
independent house/villa       168
Name: count, dtype: int64

CARPET_SQFT
PROPERTY_TYPE
residential apartment        626
independent/builder floor    568
independent house/villa       14
Name: count, dtype: int64

SUPERBUILTUP_SQFT
PROPERTY_TYPE
residential apartment        924
independent/builder floor    335
independent house/villa        3
Name: count, dtype: int64

BUILTUP_SQFT
PROPERTY_TYPE
residential apartment        211
independent/builder floor    209
independent house/villa       44
Name: count, dtype: int64

SUPER_SQFT
PROPERTY_TYPE
residential land             195
independent house/villa      102
independent/builder floor      1
Name: count, dtype: int64



## AREA column analysis

### If you want some more insights on AREA cols then go to [this notebook](../notebooks_v2/2.0_EDA.ipynb).

Also, here I am just duplicating the codes from that notebook.

In [19]:
# Extract rows which has all the areas
all_area = df[
    df["BUILTUP_SQFT"].notnull()
    & df["CARPET_SQFT"].notnull()
    & df["SUPERBUILTUP_SQFT"].notnull()
    & df["SUPER_SQFT"].notnull()
]
all_area[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
1169,https://99acres.com/A68860592,south delhi,dealer,s,independent/builder floor,17 cr,4000 sq.ft.,4000.00,0.00,0.00,0.00


In [20]:
# Rows with no area feature
no_area = df[
    df["BUILTUP_SQFT"].isnull()
    & df["CARPET_SQFT"].isnull()
    & df["SUPERBUILTUP_SQFT"].isnull()
    & df["SUPER_SQFT"].isnull()
]
no_area[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,https://99acres.com/I71649778,greater noida,dealer,s,residential apartment,1 - 1.01 cr,1272-1287 sq.ft.,NaN,NaN,NaN,NaN
1,https://99acres.com/D71227190,gurgaon,dealer,s,independent/builder floor,1 - 1.13 cr,1305-1470 sq.ft.,NaN,NaN,NaN,NaN
2,https://99acres.com/Z71649754,greater noida,dealer,s,residential apartment,1 - 1.23 cr,1365-1680 sq.ft.,NaN,NaN,NaN,NaN
3,https://99acres.com/F71380106,gurgaon,dealer,s,residential land,1 - 1.28 cr,1260-1611 sq.ft.,NaN,NaN,NaN,NaN
4,https://99acres.com/D71374796,greater noida,dealer,s,residential apartment,1 - 2.13 cr,947-2005 sq.ft.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3938,https://99acres.com/P71468812,greater noida,dealer,s,residential apartment,price on request,950-1075 sq.ft.,NaN,NaN,NaN,NaN
3965,https://99acres.com/H71394824,gurgaon,dealer,s,residential apartment,price on request,2275-4100 sq.ft.,NaN,NaN,NaN,NaN
3990,https://99acres.com/S71545876,noida,dealer,s,residential apartment,price on request,849-890 sq.ft.,NaN,NaN,NaN,NaN
3991,https://99acres.com/V71545878,noida,dealer,s,residential apartment,price on request,1322-1794 sq.ft.,NaN,NaN,NaN,NaN


In [21]:
df["CITY"].value_counts()

CITY
gurgaon          1770
greater noida     380
noida             368
south delhi       276
faridabad         211
ghaziabad         209
west delhi         77
dwarka delhi       60
east delhi         29
central delhi      27
north delhi        25
Name: count, dtype: int64

In [22]:
no_area["CITY"].value_counts()

CITY
gurgaon          399
greater noida    204
noida            176
ghaziabad         79
faridabad         57
south delhi       57
west delhi        33
dwarka delhi      17
central delhi     15
Name: count, dtype: int64

In [23]:
no_area["PRICE"].str.replace(r"\d|\W", "", regex=True).unique()

array(['cr', 'cronwards', 'l', 'lcr', 'lonwards', 'priceonrequest'],
      dtype=object)

In [24]:
no_area[["AREA", "MIN_AREA", "MAX_AREA"]].sample(7)

,AREA,MIN_AREA,MAX_AREA
2163,1160-1584 sq.ft.,1160.00,1584.00
2308,1692-3100 sq.ft.,1692.00,3100.00
3081,1225-1810 sq.ft.,1225.00,1810.00
2408,895-1125 sq.ft.,895.00,1125.00
2733,855-1275 sq.ft.,855.00,1275.00
2725,883-1232 sq.ft.,883.00,1232.00
1902,2290-2937 sq.ft.,2290.00,2937.00


In [25]:
no_area[["PRICE", "MIN_PRICE", "MAX_PRICE"]].sample(7)

,PRICE,MIN_PRICE,MAX_PRICE
2075,3.65 - 5.57 cr,36500350,55678500
2401,40 - 56.88 l,4000000,5688000
300,1.29 - 2.73 cr,12899613,27346548
2303,4.37 - 17 cr,43699425,169960175
3678,90 l - 1.62 cr,9000000,16200000
2684,50 - 61.6 l,5000000,6160000
1702,2.88 - 4.58 cr,28834217,45830397


**Observation**

- This data doesn't contains only one city's data instead it contains multiple even if I fetched data of **GURGAON city**.
- Wherever, `"-"` or `"price on request"` present in **PRICE** column they have no regular area assigned.
- But in the columns `"MIN_AREA", "MAX_AREA"` they have specified the minimum and maximum area which is written in **AREA** column.
- All of this applies for **PRICE** column too.
- So we can (sort of) estimate the price and area of these rows. But the is for those rows which are labeled as `"price on request"` in **PRICE** column. They have the AREA but not the PRICE.

In [26]:
# Properties which has all three area present
b_c_sb = df[
    df["BUILTUP_SQFT"].notnull()
    & df["CARPET_SQFT"].notnull()
    & df["SUPERBUILTUP_SQFT"].notnull()
]
b_c_sb[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
26,https://99acres.com/Z71180896,faridabad,dealer,s,independent/builder floor,1.01 cr,1800 sq.ft.,1600.00,1800.00,1700.00,NaN
58,https://99acres.com/G69013118,north delhi,dealer,s,independent/builder floor,1.05 cr,850 sq.ft.,800.00,850.00,830.00,NaN
60,https://99acres.com/K70841634,ghaziabad,dealer,s,residential apartment,1.05 cr,1585 sq.ft.,1110.00,1585.00,1261.00,NaN
66,https://99acres.com/D70842394,ghaziabad,dealer,s,residential apartment,1.06 cr,1585 sq.ft.,1272.00,1585.00,1428.00,NaN
67,https://99acres.com/O70645794,faridabad,dealer,s,independent/builder floor,1.06 cr,2250 sq.ft.,1850.00,2250.00,2150.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3728,https://99acres.com/M70330948,faridabad,dealer,s,independent/builder floor,95 l,1800 sq.ft.,1780.00,1800.00,1790.00,NaN
3734,https://99acres.com/J57570924,gurgaon,dealer,r,residential apartment,"95,000",2800 sq.ft.,2790.00,2800.00,2795.00,NaN
3735,https://99acres.com/H70701912,gurgaon,dealer,r,independent/builder floor,"95,000",2250 sq.ft.,1850.00,2250.00,2059.00,NaN
3754,https://99acres.com/M70066816,faridabad,dealer,s,independent/builder floor,98 l,2250 sq.ft.,1800.00,2250.00,2070.00,NaN


In [27]:
def calc_ratio(df: pd.DataFrame, f1: str, f2: str) -> None:
    ratio = np.round(
        df[f1].div(df[f2].replace(0, df["BUILTUP_SQFT"].quantile(0.9))).mean(), 3
    )
    print(f"Ration b/w {f1} and {f2}:".rjust(50), ratio)
    print(f"Ration b/w {f2} and {f1}:".rjust(50), np.round(1 / ratio, 3))

In [28]:
calc_ratio(b_c_sb, "CARPET_SQFT", "BUILTUP_SQFT")
calc_ratio(b_c_sb, "SUPERBUILTUP_SQFT", "BUILTUP_SQFT")

          Ration b/w CARPET_SQFT and BUILTUP_SQFT: 0.906
          Ration b/w BUILTUP_SQFT and CARPET_SQFT: 1.104
    Ration b/w SUPERBUILTUP_SQFT and BUILTUP_SQFT: 1.102
    Ration b/w BUILTUP_SQFT and SUPERBUILTUP_SQFT: 0.907


In [29]:
# Properties which has builtup and carpet area present
b_c = df[df["BUILTUP_SQFT"].notnull() & df["CARPET_SQFT"].notnull()]
b_c[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
5,https://99acres.com/S69490898,west delhi,owner,s,independent/builder floor,1 cr,1350 sq.ft.,1210.00,NaN,1350.00,NaN
6,https://99acres.com/S70727184,faridabad,dealer,s,independent/builder floor,1 cr,2115 sq.ft.,2100.00,NaN,2115.00,NaN
26,https://99acres.com/Z71180896,faridabad,dealer,s,independent/builder floor,1.01 cr,1800 sq.ft.,1600.00,1800.00,1700.00,NaN
54,https://99acres.com/L70726694,faridabad,dealer,s,independent/builder floor,1.05 cr,2970 sq.ft.,2900.00,NaN,2970.00,NaN
58,https://99acres.com/G69013118,north delhi,dealer,s,independent/builder floor,1.05 cr,850 sq.ft.,800.00,850.00,830.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3734,https://99acres.com/J57570924,gurgaon,dealer,r,residential apartment,"95,000",2800 sq.ft.,2790.00,2800.00,2795.00,NaN
3735,https://99acres.com/H70701912,gurgaon,dealer,r,independent/builder floor,"95,000",2250 sq.ft.,1850.00,2250.00,2059.00,NaN
3754,https://99acres.com/M70066816,faridabad,dealer,s,independent/builder floor,98 l,2250 sq.ft.,1800.00,2250.00,2070.00,NaN
3755,https://99acres.com/U70786592,gurgaon,dealer,s,independent/builder floor,98 l,1550 sq.ft.,1250.00,1550.00,1450.00,NaN


In [30]:
calc_ratio(b_c, "CARPET_SQFT", "BUILTUP_SQFT")

          Ration b/w CARPET_SQFT and BUILTUP_SQFT: 0.905
          Ration b/w BUILTUP_SQFT and CARPET_SQFT: 1.105


In [31]:
# Properties which has builtup and super_builtup area present
b_sb = df[df["BUILTUP_SQFT"].notnull() & df["SUPERBUILTUP_SQFT"].notnull()]
b_sb[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
26,https://99acres.com/Z71180896,faridabad,dealer,s,independent/builder floor,1.01 cr,1800 sq.ft.,1600.00,1800.00,1700.00,NaN
58,https://99acres.com/G69013118,north delhi,dealer,s,independent/builder floor,1.05 cr,850 sq.ft.,800.00,850.00,830.00,NaN
60,https://99acres.com/K70841634,ghaziabad,dealer,s,residential apartment,1.05 cr,1585 sq.ft.,1110.00,1585.00,1261.00,NaN
66,https://99acres.com/D70842394,ghaziabad,dealer,s,residential apartment,1.06 cr,1585 sq.ft.,1272.00,1585.00,1428.00,NaN
67,https://99acres.com/O70645794,faridabad,dealer,s,independent/builder floor,1.06 cr,2250 sq.ft.,1850.00,2250.00,2150.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3728,https://99acres.com/M70330948,faridabad,dealer,s,independent/builder floor,95 l,1800 sq.ft.,1780.00,1800.00,1790.00,NaN
3734,https://99acres.com/J57570924,gurgaon,dealer,r,residential apartment,"95,000",2800 sq.ft.,2790.00,2800.00,2795.00,NaN
3735,https://99acres.com/H70701912,gurgaon,dealer,r,independent/builder floor,"95,000",2250 sq.ft.,1850.00,2250.00,2059.00,NaN
3754,https://99acres.com/M70066816,faridabad,dealer,s,independent/builder floor,98 l,2250 sq.ft.,1800.00,2250.00,2070.00,NaN


In [32]:
calc_ratio(b_sb, "SUPERBUILTUP_SQFT", "BUILTUP_SQFT")

    Ration b/w SUPERBUILTUP_SQFT and BUILTUP_SQFT: 1.107
    Ration b/w BUILTUP_SQFT and SUPERBUILTUP_SQFT: 0.903


In [33]:
# Properties which has builtup and super_area (plot area) area present
b_su = df[df["BUILTUP_SQFT"].notnull() & df["SUPER_SQFT"].notnull()]
b_su[imp_cols]

,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
197,https://99acres.com/L17074947,gurgaon,owner,s,independent house/villa,1.2 cr,60 sq.ft.,1350.00,NaN,1450.00,60.00
453,https://99acres.com/K71519744,gurgaon,dealer,r,independent house/villa,1.49 l,2700 sq.ft.,4950.00,NaN,5000.00,2700.00
837,https://99acres.com/O71072290,gurgaon,dealer,s,independent house/villa,10 cr,4950 sq.ft.,5400.00,NaN,6800.00,4950.00
1038,https://99acres.com/Y71054116,gurgaon,dealer,s,independent house/villa,12.5 cr,3150 sq.ft.,NaN,NaN,58680.00,3150.00
1111,https://99acres.com/X70832318,south delhi,dealer,s,independent house/villa,15 cr,10000 sq.ft.,NaN,NaN,9000.00,10000.00
1169,https://99acres.com/A68860592,south delhi,dealer,s,independent/builder floor,17 cr,4000 sq.ft.,4000.00,0.00,0.00,0.00
1648,https://99acres.com/T70715086,gurgaon,dealer,s,independent house/villa,2.73 cr,320 sq.ft.,NaN,NaN,1868.00,320.00
2103,https://99acres.com/J57975272,gurgaon,owner,s,independent house/villa,3.8 cr,1623 sq.ft.,NaN,NaN,2299.95,1623.15
2568,https://99acres.com/X70075340,gurgaon,dealer,s,independent house/villa,5.1 cr,3078 sq.ft.,900.00,NaN,1125.00,3078.00
2692,https://99acres.com/L71134906,noida,dealer,r,independent house/villa,"50,000",4305 sq.ft.,NaN,NaN,4305.57,4305.57


There is a problem in `https://99acres.com/Q69692678` property.

In [34]:
calc_ratio(b_su, "SUPER_SQFT", "BUILTUP_SQFT")
calc_ratio(b_su, "BUILTUP_SQFT", "SUPER_SQFT")

           Ration b/w SUPER_SQFT and BUILTUP_SQFT: 0.73
           Ration b/w BUILTUP_SQFT and SUPER_SQFT: 1.37
           Ration b/w BUILTUP_SQFT and SUPER_SQFT: 5.146
           Ration b/w SUPER_SQFT and BUILTUP_SQFT: 0.194


## How to encode `BUILTUP_SQFT` column?

**Things to Remember**

- Encode **`BUILTUP_SQFT`** according to `PREFERENCE`, `PROPERTY_TYPE` and `CITY`.
  - **`PREFERENCE`:** Maybe you can eliminate the `"p"` tpye PREFERENCE, due to its PRICE values.
  - **`PROPERTY_TYPE`:** Use only following _independent house/villa, independent/builder floor, residential apartment, residential land_ PROPERTY_TYPE.
  - **`CITY`:** Calculate `BUILTUP_SQFT` values for corresponding CITY.

**How to Calculate `BUILTUP_SQFT`**

1. `groupby()`: CITY + PREFERENCE + PROPERTY_TYPE (exclude: `"p"`)
2. `merge()`: Merge the different dataframes into one.
3. `mean()`: Calculate avegrage for all the combination forms while `groupby` step.

In [35]:
t1 = (
    df.query("PREFERENCE!='p'")[imp_cols]
    .groupby(["CITY", "PREFERENCE", "PROPERTY_TYPE"])[area_cols[1:]]
    .mean()
    .reset_index()
    .fillna(0)
)
print(t1.shape)
t1.head()

(57, 7)


,CITY,PREFERENCE,PROPERTY_TYPE,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,central delhi,s,independent/builder floor,1800.00,0.00,0.00,0.00
1,central delhi,s,residential apartment,2182.90,3740.00,0.00,0.00
2,dwarka delhi,r,residential apartment,0.00,1131.00,0.00,0.00
3,dwarka delhi,s,independent/builder floor,1469.58,1741.67,1698.17,0.00
4,dwarka delhi,s,residential apartment,1882.86,2685.71,2000.00,0.00


In [36]:
t2 = (
    b_c.query("PREFERENCE!='p'")[imp_cols]
    .groupby(["CITY", "PREFERENCE", "PROPERTY_TYPE"])[area_cols[1:]]
    .mean()
    .reset_index()
    .fillna(0)
)
print(t2.shape)
t2.head()

(27, 7)


,CITY,PREFERENCE,PROPERTY_TYPE,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,dwarka delhi,s,independent/builder floor,1649.09,1840.91,1740.45,0.00
1,dwarka delhi,s,residential apartment,1600.00,2000.00,1800.00,0.00
2,east delhi,s,residential apartment,1469.71,1840.00,1600.14,0.00
3,faridabad,s,independent/builder floor,2109.32,2338.83,2253.83,0.00
4,faridabad,s,residential apartment,1580.20,1800.40,1683.20,0.00


In [37]:
# Not a good approach
pd.merge(t1, t2, "outer", ["CITY", "PREFERENCE", "PROPERTY_TYPE"]).head()

,CITY,PREFERENCE,PROPERTY_TYPE,CARPET_SQFT_x,SUPERBUILTUP_SQFT_x,BUILTUP_SQFT_x,SUPER_SQFT_x,CARPET_SQFT_y,SUPERBUILTUP_SQFT_y,BUILTUP_SQFT_y,SUPER_SQFT_y
0,central delhi,s,independent/builder floor,1800.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
1,central delhi,s,residential apartment,2182.90,3740.00,0.00,0.00,NaN,NaN,NaN,NaN
2,dwarka delhi,r,residential apartment,0.00,1131.00,0.00,0.00,NaN,NaN,NaN,NaN
3,dwarka delhi,s,independent/builder floor,1469.58,1741.67,1698.17,0.00,1649.09,1840.91,1740.45,0.00
4,dwarka delhi,s,residential apartment,1882.86,2685.71,2000.00,0.00,1600.00,2000.00,1800.00,0.00


In [38]:
groupby_cols = ["CITY", "PREFERENCE", "PROPERTY_TYPE", "CLASS_LABEL"]

In [39]:
def apply_groupby(*df: pd.DataFrame) -> list[pd.DataFrame]:
    to_return = []
    for i in df:
        grp_df = i.groupby(groupby_cols)[area_cols[1:]].mean().reset_index().fillna(0)
        to_return.append(grp_df)
    return to_return

In [40]:
def calculate_average_between_similar_columns(df: pd.DataFrame) -> pd.DataFrame:
    avg_df = pd.DataFrame()
    for norm_col in imp_area_cols:
        avg_df[norm_col] = (
            df.loc[:, [i for i in df.columns if norm_col in i]].mean(axis=1).round(2)
        )

    return_df = pd.concat([df[groupby_cols], avg_df], axis=1)
    return return_df

In [41]:
def merge_multiple_df(*df: pd.DataFrame) -> pd.DataFrame:
    merged_df = df[0]
    for i, _df in enumerate(df[1:], 1):
        merged_df = pd.merge(
            merged_df,
            _df,
            "outer",
            groupby_cols,
            suffixes=[f"_{i}", f"_{i+1}"],
        )
    return merged_df

In [42]:
temp = merge_multiple_df(*apply_groupby(df, b_c, b_c_sb, b_sb, b_su))
print(temp.shape)
temp.head()

(105, 24)


,CITY,PREFERENCE,PROPERTY_TYPE,CLASS_LABEL,CARPET_SQFT_1,SUPERBUILTUP_SQFT_1,BUILTUP_SQFT_1,SUPER_SQFT_1,CARPET_SQFT_2,SUPERBUILTUP_SQFT_2,...,BUILTUP_SQFT_3,SUPER_SQFT_3,CARPET_SQFT_4,SUPERBUILTUP_SQFT_4,BUILTUP_SQFT_4,SUPER_SQFT_4,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,central delhi,s,independent/builder floor,owner,1800.00,0.00,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,central delhi,s,residential apartment,dealer,2092.11,3740.00,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,central delhi,s,residential apartment,owner,3000.00,0.00,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dwarka delhi,r,residential apartment,dealer,0.00,1800.00,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dwarka delhi,r,residential apartment,owner,0.00,462.00,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
avg_df = calculate_average_between_similar_columns(temp)
avg_df.head()

,CITY,PREFERENCE,PROPERTY_TYPE,CLASS_LABEL,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,central delhi,s,independent/builder floor,owner,1800.00,0.00,0.00,0.00
1,central delhi,s,residential apartment,dealer,2092.11,3740.00,1870.00,0.00
2,central delhi,s,residential apartment,owner,3000.00,0.00,0.00,0.00
3,dwarka delhi,r,residential apartment,dealer,0.00,1800.00,900.00,0.00
4,dwarka delhi,r,residential apartment,owner,0.00,462.00,231.00,0.00


## Fill NaN values present in AREA columns

In [44]:
def fillna_with_average(avg_df: pd.DataFrame, main_df: pd.DataFrame) -> pd.DataFrame:
    for i in avg_df.itertuples(False):
        temp = main_df.query(
            "PREFERENCE==@i.PREFERENCE "
            "and PROPERTY_TYPE==@i.PROPERTY_TYPE "
            "and CITY==@i.CITY "
            "and CLASS_LABEL==@i.CLASS_LABEL "
        ).fillna({k: getattr(i, k) for k in imp_area_cols})
        main_df.loc[temp.index, imp_area_cols] = temp[imp_area_cols]
    return main_df

In [45]:
temp = fillna_with_average(avg_df, df[imp_cols])
print(temp.shape)
temp.head()

(3432, 11)


,PROP_ID,CITY,CLASS_LABEL,PREFERENCE,PROPERTY_TYPE,PRICE,AREA,CARPET_SQFT,SUPERBUILTUP_SQFT,BUILTUP_SQFT,SUPER_SQFT
0,https://99acres.com/I71649778,greater noida,dealer,s,residential apartment,1 - 1.01 cr,1272-1287 sq.ft.,1133.44,1636.48,1399.66,0.00
1,https://99acres.com/D71227190,gurgaon,dealer,s,independent/builder floor,1 - 1.13 cr,1305-1470 sq.ft.,2147.53,2678.89,2554.87,0.00
2,https://99acres.com/Z71649754,greater noida,dealer,s,residential apartment,1 - 1.23 cr,1365-1680 sq.ft.,1133.44,1636.48,1399.66,0.00
3,https://99acres.com/F71380106,gurgaon,dealer,s,residential land,1 - 1.28 cr,1260-1611 sq.ft.,0.00,0.00,0.00,2738.08
4,https://99acres.com/D71374796,greater noida,dealer,s,residential apartment,1 - 2.13 cr,947-2005 sq.ft.,1133.44,1636.48,1399.66,0.00


**Note**

1. As you know the same thing is done by Nitish sir in a different way.
2. I think this method is also a good approach and reliable.
3. Keep in mind, that you have to use **`SUPER_AREA`** where it is present.
4. In my approach I used only **`BUILTUP_SQFT`** column to impute itself but sir uses other columns. (How do I do do that?)
